In [ ]:
import sys
sys.path.insert(0, '../') #to load FileDataGenerator

In [ ]:
from FileDataGenerator import FileDataGen
import os
import numpy as np

In [10]:
from keras import Input, optimizers
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras.models import Model

In [11]:
def get_model():
    
    entrada= Input(shape=(150,150,3))
    
    conv = Conv2D(filters=32, kernel_size=3, activation='relu', name='conv_1')(entrada)
    maxpool = MaxPool2D(pool_size=2, strides=2, name='maxpool_1')(conv)
    
    conv = Conv2D(filters=64, kernel_size=3, activation='relu', name='conv_2')(maxpool)
    maxpool = MaxPool2D(pool_size=2, strides=2, name='maxpool_2')(conv)   
    
    conv = Conv2D(filters=128, kernel_size=3, activation='relu', name='conv_3')(maxpool)
    maxpool = MaxPool2D(pool_size=2, strides=2, name='maxpool_3')(conv)
        
    conv = Conv2D(filters=128, kernel_size=3, activation='relu', name='conv_4')(maxpool)
    maxpool = MaxPool2D(pool_size=2, strides=2, name='maxpool_4')(conv)
    
    flat = Flatten(name='flatten')(maxpool)
    #drop = Dropout(rate=.5, name='dropout')(flat)
    
    dense = Dense(units=512, activation='relu', name='Dense1')(flat)#(drop)
    output = Dense(units=1, activation='sigmoid', name='output')(dense)
    #output = Dense(units=2, activation='softmax', name='output')(dense)
    
    model = Model(entrada, output)
    
    #model.compile(optimizer=optimizers.RMSprop(lr=0.001), loss='categorical_crossentropy', metrics=['acc'])
    model.compile(optimizer=optimizers.RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])
    
    return model

In [2]:
train_Path = '/Users/dfreire/Dropbox/Datasets/small_dataset/train'
val_Path = '/Users/dfreire/Dropbox/Datasets/small_dataset/validation'

In [3]:
def Load_Directory(DB_path):
    #Read each folder
    data=[]
    labels=[]
    for class_ in os.listdir(DB_path):
        dat = [os.path.join(DB_path, class_, img) for img in os.listdir(os.path.join(DB_path, class_))]
        lab=[class_ for i in os.listdir(os.path.join(DB_path, class_))]
        labels = labels+lab
        data = data + dat
    return np.array(data), np.array(labels)

In [4]:
x_train, y_train = Load_Directory(train_Path)
x_val, y_val = Load_Directory(val_Path)

In [5]:
train_datagen = FileDataGen(rescale=1./255) 
val_datagen = FileDataGen(rescale=1./255) 

In [6]:
train_gen = train_datagen.flow_from_filelist(x_train,y_train,
                                             target_size=(150,150),
                                             batch_size=20,
                                             class_mode='binary')
val_gen = val_datagen.flow_from_filelist(x_val,y_val,
                                             target_size=(150,150),
                                             batch_size=20,
                                             class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [7]:
for data, label in train_gen:
    print(data.shape)
    print(label.shape)
    break

(20, 150, 150, 3)
(20,)


In [14]:
model = get_model()

In [15]:
h = model.fit_generator(train_gen, 
                       steps_per_epoch = 100,
                       epochs= 10,
                       validation_data=val_gen,
                       validation_steps=50)

Epoch 1/10
100/100 [==============================] - 111s 1s/step - loss: 0.7761 - acc: 0.5035 - val_loss: 0.6917 - val_acc: 0.5140
Epoch 2/10
100/100 [==============================] - 108s 1s/step - loss: 0.6933 - acc: 0.5640 - val_loss: 0.6718 - val_acc: 0.5720
Epoch 3/10
100/100 [==============================] - 107s 1s/step - loss: 0.6625 - acc: 0.6250 - val_loss: 0.6546 - val_acc: 0.5730
Epoch 4/10
100/100 [==============================] - 108s 1s/step - loss: 0.6306 - acc: 0.6675 - val_loss: 0.6259 - val_acc: 0.6710
Epoch 5/10
100/100 [==============================] - 107s 1s/step - loss: 0.5784 - acc: 0.7065 - val_loss: 0.6077 - val_acc: 0.6540
Epoch 6/10
100/100 [==============================] - 108s 1s/step - loss: 0.5552 - acc: 0.7205 - val_loss: 0.5911 - val_acc: 0.6920
Epoch 7/10
100/100 [==============================] - 108s 1s/step - loss: 0.5224 - acc: 0.7515 - val_loss: 0.6054 - val_acc: 0.6950
Epoch 8/10
100/100 [==============================] - 108s 1s/step - 

In [16]:
model.save('cats_dogs_small_filegen.h5')